In [9]:
import carla 
import math 
import time    
import random 
import numpy as np 
import cv2 

client = carla.Client('localhost', 2000) 
client.reload_world() 
world = client.get_world() 

bp_lib = world.get_blueprint_library() 
spawn_points = world.get_map().get_spawn_points() 

In [10]:
vehicle_bp = bp_lib.find('vehicle.lincoln.mkz_2020') 
vehicle = world.try_spawn_actor(vehicle_bp, spawn_points[79]) 

spectator = world.get_spectator() 
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4, z=2.5)), vehicle.get_transform().rotation) 
spectator.set_transform(transform) 

for i in range(200): 
    vehicle_bp = random.choice(bp_lib.filter('vehicle')) 
    npc = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points)) 


In [11]:
for v in world.get_actors().filter('*vehicle*'): 
    v.set_autopilot(True) 


In [12]:
camera_bp = bp_lib.find('sensor.camera.rgb') 
camera_init_trans = carla.Transform(carla.Location(z=2)) 
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle) 

gnss_bp = bp_lib.find('sensor.other.gnss') 
gnss = world.spawn_actor(gnss_bp, carla.Transform(), attach_to=vehicle) 

imu_bp = bp_lib.find('sensor.other.imu') 
imu = world.spawn_actor(imu_bp, carla.Transform(), attach_to=vehicle) 

collision_bp = bp_lib.find('sensor.other.collision') 
collision = world.spawn_actor(collision_bp, carla.Transform(), attach_to=vehicle) 

lane_inv_bp = bp_lib.find('sensor.other.lane_invasion') 
lane_inv = world.spawn_actor(lane_inv_bp, carla.Transform(), attach_to=vehicle) 

obstacle_bp = bp_lib.find('sensor.other.obstacle') 
obstacle_bp.set_attribute('hit_radius', '0.5')
obstacle_bp.set_attribute('distance', '50') 
obstacle_sensor = world.spawn_actor(obstacle_bp, carla.Transform(), attach_to=vehicle) 

In [13]:
def build_projection_matrix(w, h, fov): 
    focal = w / (2.0 * np.tan(fov * np.pi / 360.0)) 
    K = np.identity(3) 
    K[0, 0] = K[1, 1] = focal
    K[0, 2] = w / 2.0 
    K[1, 2] = h / 2.0 
    return K 


def get_image_point(loc, K, w2c): 
    # Calculate 2D projection of 3D coordinate 

    # Format the input coordinate (loc is a carla.Position object)_ 
    point = np.array([loc.x, loc.y, loc.z, 1]) 
    # transform to camera coordinates 
    point_camera = np.dot(w2c, point) 

    # New we must change from UE4's coordinate system to an "standard" 
    # (x, y, z) -> (y, -z, x) 
    # and we remove the fourth componebonent also 
    point_camera = [point_camera[1], -point_camera[2], point_camera[0]] 
    
    # now project 3D -> 2D using the camera matrix 
    point_img = np.dot(K, point_camera) 
    # normalize  
    point_img[0] /= point_img[2] 
    point_img[1] /= point_img[2] 

    return tuple(map(int, point_img[:2])) 


world_2_camera = np.array(camera.get_transform().get_inverse_matrix()) 

# Get the attributes from the camera 
image_w = camera_bp.get_attribute("image_size_x").as_int() 
image_h = camera_bp.get_attribute("image_size_y").as_int() 
fov = camera_bp.get_attribute("fov").as_float() 

# calculate the camera projection matrix to project from 3D -> 2D 
K = build_projection_matrix(image_w, image_h, fov) 

In [14]:


def rgb_callback(image, data_dict): 
    data_dict['rgb_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4)) 

def gnss_callback(data, data_dict): 
    data_dict['gnss'] = [data.latitude, data.longitude]  # data.altitude 

def imu_callback(data, data_dict):
    data_dict['imu'] = {
        'gyro': data.gyroscope, 
        'accel': data.accelerometer, 
        'compass': data.compass 
    }

def lane_inv_callback(event, data_dict): 
    data_dict['lane_invasion'] == True  

def collision_callback(event, data_dict): 
    data_dict['collision'] = True 

def obstacle_callback(event, data_dict, camera, k_mat): 
    if 'static' not in event.other_actor.type_id: 
        data_dict['obstacle'].append({'transform': event.other_actor.type_id, 'frame': event.frame}) 

    world_2_camera = np.array(camera.get_transform().get_inverse_matrix()) 
    image_point = get_image_point(event.other_actor.get_transform().location, k_mat, world_2_camera) 
    if 0 < image_point[0] < image_w and 0 < image_point[1] < image_h: 
        cv2.circle(data_dict['rgb_image'], tuple(image_point), 10, (0, 0, 255), 3)

In [15]:
# Display the image in agnss_sensorCV display window 
image_w = camera_bp.get_attribute("image_size_x").as_int() 
image_h = camera_bp.get_attribute("image_size_y").as_int() 

collision_counter = 20 
lane_invasion_counter = 20 

sensor_data = {'rgb_image': np.zeros((image_h, image_w, 4)), 
               'lane_invasion': False, 
               'collision': False,
               'gnss': [0,0],  
               'obstacle': [], 
               'imu': {
                   'gyro': carla.Vector3D(), 
                   'accel': carla.Vector3D(), 
                   'compass': 0
               }}

cv2.namedWindow('RGB Camera', cv2.WINDOW_NORMAL) 

cv2.imshow('RGB Camera', sensor_data['rgb_image']) 
cv2.waitKey(1) 

camera.listen(lambda image: rgb_callback(image, sensor_data)) 
collision.listen(lambda event: collision_callback(event, sensor_data)) 
gnss.listen(lambda event: gnss_callback(event, sensor_data)) 
imu.listen(lambda event: imu_callback(event, sensor_data))  
lane_inv.listen(lambda event: lane_inv_callback(event, sensor_data)) 
obstacle_sensor.listen(lambda event: obstacle_callback(event, sensor_data, camera, K)) 

font = cv2.FONT_HERSHEY_SIMPLEX 
bottomLeftCornerOfText = (10, 50) 
fontScale = 0.5 
fontColor = (255, 255, 255) 
lineType = 2 
thickness = 2 


def draw_compass(img, theta): 
    
    compass_center = (700, 100) 
    compass_size = 50 

    cardinal_directions = [
        ('N', [0, -1]), 
        ('E', [1, 0]), 
        ('S', [0, 1]), 
        ('W', [-1, 0])
    ]

    for car_dir in cardinal_directions: 
        cv2.putText(sensor_data['rgb_image'], car_dir[0], 
                    (int(compass_center[0] + 1.2 * compass_size * car_dir[1][0]), int(compass_center[1] + 1.2 * compass_size * car_dir[1][1])), 
                    font, fontScale, fontColor, lineType)
        
        compass_point = (int(compass_center[0] + compass_size * math.sin(theta)), int(compass_center[1] - compass_size * math.cos(theta))) 
        cv2.line(img, compass_center, compass_point, (255, 255, 255), 3) 


while True: 
    cv2.putText(sensor_data['rgb_image'], 'Lat: ' + str(sensor_data['gnss'][0]), (10, 30), font, fontScale, fontColor, thickness, lineType) 
    cv2.putText(sensor_data['rgb_image'], 'Lon: ' + str(sensor_data['gnss'][1]), (10, 50), font, fontScale, fontColor, thickness, lineType) 
    accel = sensor_data['imu']['accel'] - carla.Vector3D(x=0, y=0, z=9.8) 
    cv2.putText(sensor_data['rgb_image'], 'Accel: ' + str(accel), (10, 70), font, fontScale, fontColor, thickness, lineType)
    cv2.putText(sensor_data['rgb_image'], 'Gyro: ' + str(sensor_data['imu']['gyro']), (10, 100), font, fontScale, fontColor, thickness, lineType)  
    cv2.putText(sensor_data['rgb_image'], 'Compass: ' + str(sensor_data['imu']['compass']), (10, 120), font, fontScale, fontColor, thickness, lineType) 

    draw_compass(sensor_data['rgb_image'], sensor_data['imu']['compass']) 

    if sensor_data['collision']: 
        collision_counter -= 1 
        if collision_counter <= 1: 
            sensor_data['collision'] = False 
        cv2.putText(sensor_data['rgb_image'], 'COLLISION', (250, 300), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3, 2) 
    else: 
        collision_counter = 20 

    if sensor_data['lane_invasion']: 
        lane_invasion_counter -= 1 
        if lane_invasion_counter <= 1: 
            sensor_data['lane_invasion'] = False 
        cv2.putText(sensor_data['rgb_image'], 'LANE INVASION', (190, 350), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3, 2) 
    else: 
        lane_invasion_counter = 20 

    cv2.imshow('RGB Camera', sensor_data['rgb_image']) 
    # Break the loop if the user presses the 0 key 
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break 


# close the opencv display window when the game loop stops 
camera.stop() 
collision.stop() 
gnss.stop() 
imu.stop() 
lane_inv.stop() 
obstacle_sensor.stop() 
cv2.destroyAllWindows() 






: 